# TLM Ports and Exports

Note: These examples must be run in a freshly restarted notebook kernal.  Use the >> icon above to run the example.

## Mailbox Example

In [1]:
from pyuvm import *
import random

class POBox(uvm_nonblocking_get_export):
    def try_get(self):
        if random.randint(0,3) == 0:
            return "Congratulations", True
        else:
            return None, False

class PostOffice(uvm_component):
    def build_phase(self):
        self.po_box_export = POBox("po_box_export", self)

class CompulsiveChecker(uvm_component):
    def build_phase(self):
        self.po_box_port  = uvm_nonblocking_get_port("po_box_port", self)

    def run_phase(self):
        self.raise_objection()
        got_mail = False
        while not got_mail:
            print("checking mail and got . . .", end=" ")
            mail, got_mail = self.po_box_port.try_get()
            print(mail)
        self.drop_objection()
        
class Community(uvm_test):
    def build_phase(self):
        self.post_office  = PostOffice("post_office", self)
        self.mail_checker = CompulsiveChecker("mail_checker", self)
    
    def connect_phase(self):
        self.mail_checker.po_box_port.connect(self.post_office.po_box_export)
        
uvm_root().run_test("Community")

checking mail and got . . . None
checking mail and got . . . None
checking mail and got . . . Congratulations
